In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load data into a pandas DataFrame
df = pd.read_csv("year_2013_2022.csv")
df['Infant'] = df[['< 1 yr',' 1 yr','2 yrs']].sum(axis=1)
df['Children'] = df.loc[:,'3 yrs':'14 yrs'].sum(axis=1)
df['Youth'] = df.loc[:,'15 yrs':'24 yrs'].sum(axis=1)
df['Adult'] = df.loc[:,'25 yrs':'64 yrs'].sum(axis=1)
df['Elderly'] = df.loc[:,'65 yrs':'100 yrs'].sum(axis=1) + df['> 100 yrs']
df.rename(columns={'ผลรวมประชากรทั้งหมด':'Total_Populations'},inplace=True)
AgeGroups = ['Infant','Children','Youth','Adult','Elderly']

df['Total_Populations'] = df['Total_Populations'].str.replace(',','')
df['Total_Populations'] = df['Total_Populations'].astype('float')

df.head()

,ปีเดือน,รหัสจังหวัด,ชื่อจังหวัด,รหัสสำนักทะเบียน,ชื่อสำนักทะเบียน,รหัสตำบล,ชื่อตำบล,รหัสหมู่บ้าน,ชื่อหมู่บ้าน,ชาย < 1 ปี,...,96 yrs,97 yrs,98 yrs,99 yrs,100 yrs,Infant,Children,Youth,Adult,Elderly
0,5612,0.0,ทั่วประเทศ,0.0,NaN,0.0,NaN,0.0,NaN,358688,...,9739,7917,6663,4674,4007,2255002,9549486,9510850,36485907,5849348
1,5612,10.0,กรุงเทพมหานคร,0.0,NaN,0.0,NaN,0.0,NaN,24126,...,923,817,645,551,445,154752,749974,787831,3290395,534703
2,5612,11.0,สมุทรปราการ,0.0,NaN,0.0,NaN,0.0,NaN,6976,...,196,150,118,119,75,43225,180767,173694,731024,94202
3,5612,12.0,นนทบุรี,0.0,NaN,0.0,NaN,0.0,NaN,5611,...,280,241,193,166,162,35615,153774,149103,692330,110002
4,5612,13.0,ปทุมธานี,0.0,NaN,0.0,NaN,0.0,NaN,5844,...,106,83,77,53,53,37209,157011,150726,621848,73811


In [2]:
total_pops = df[df['ชื่อจังหวัด']== 'ทั่วประเทศ'][['ชื่อจังหวัด','year','Total_Populations','Infant','Children','Youth','Adult','Elderly']].sort_values('year')

In [3]:
years = []
for i in range(2013,2051):
    years.append(i)
    # X_poly_pred = poly.fit_transform(X_pred)


In [4]:
import warnings
warnings.filterwarnings('ignore')

# total_pops['year'] = pd.to_datetime(total_pops['year'])
# total_pops.set_index('year', inplace=True)
ExpoSmoothPredTotal = pd.DataFrame()
# define the features and target
for Province in df['ชื่อจังหวัด'].drop_duplicates():
    ExpoSmoothPred = pd.DataFrame(years, columns = ["year"])
    total_pops = df[df['ชื่อจังหวัด']== Province][['ชื่อจังหวัด','year','Total_Populations','Infant','Children','Youth','Adult','Elderly']].sort_values('year')


    for AgeGroup in AgeGroups:
        y = total_pops[AgeGroup]
        # Set the date column as the index

        # Fit the model
        model = ExponentialSmoothing(y,trend='add').fit()

        # Make predictions
        predictions = model.predict(start=0, end=len(total_pops)+27)
        ExpoSmoothPred[AgeGroup] = predictions
        

    ExpoSmoothPred['Sum Prediction'] = ExpoSmoothPred.loc[:,"Infant":"Elderly"].sum(axis=1)
    #     ExpoSmoothPred.plot.line(x='year',y='Predicted '+AgeGroup+' Population')
    #     plt.show()
    y = total_pops['Total_Populations']
    # Set the date column as the index

    # Fit the model
    model = ExponentialSmoothing(y,trend='add').fit()

    # Make predictions
    predictions = model.predict(start=0, end=len(total_pops)+27)
    ExpoSmoothPred['Total_Populations'] = predictions
    ExpoSmoothPred.clip(lower=0, inplace = True)
    ExpoSmoothPred['Province'] = Province
    ExpoSmoothPredTotal = pd.concat([ExpoSmoothPredTotal,ExpoSmoothPred])

In [5]:
ExpoSmoothPredTotal.Province.nunique()

78

In [6]:
ExpoSmoothPredTotal

,year,Infant,Children,Youth,Adult,Elderly,Sum Prediction,Total_Populations,Province
0,2013,2.208311e+06,9.490988e+06,9.426862e+06,3.670468e+07,6.123509e+06,6.395435e+07,6.512472e+07,ทั่วประเทศ
1,2014,2.232291e+06,9.540494e+06,9.497714e+06,3.666452e+07,6.124853e+06,6.405987e+07,6.498266e+07,ทั่วประเทศ
2,2015,2.173518e+06,9.443150e+06,9.354376e+06,3.688957e+07,6.397649e+06,6.425826e+07,6.537933e+07,ทั่วประเทศ
3,2016,2.035985e+06,9.408476e+06,9.198821e+06,3.715103e+07,6.672709e+06,6.446703e+07,6.613108e+07,ทั่วประเทศ
4,2017,1.955335e+06,9.358071e+06,8.980469e+06,3.739005e+07,6.931164e+06,6.461509e+07,6.625162e+07,ทั่วประเทศ
...,...,...,...,...,...,...,...,...,...
33,2046,1.778941e+04,1.294293e+05,1.181983e+05,5.527583e+05,1.059091e+05,9.240844e+05,9.019286e+05,นราธิวาส
34,2047,1.708788e+04,1.281334e+05,1.176492e+05,5.585134e+05,1.074795e+05,9.288634e+05,9.055960e+05,นราธิวาส
35,2048,1.638635e+04,1.268374e+05,1.171002e+05,5.642685e+05,1.090499e+05,9.336423e+05,9.092634e+05,นราธิวาส
36,2049,1.568482e+04,1.255415e+05,1.165511e+05,5.700236e+05,1.106202e+05,9.384213e+05,9.129307e+05,นราธิวาส


In [7]:
ExpoSmoothPredTotal['DependencyRatio'] = ( ExpoSmoothPredTotal['Infant'] + ExpoSmoothPredTotal['Children']+ ExpoSmoothPredTotal['Elderly'])/(ExpoSmoothPredTotal['Youth'] + ExpoSmoothPredTotal['Adult'])  

In [8]:
ExpoSmoothPredTotal.head()

,year,Infant,Children,Youth,Adult,Elderly,Sum Prediction,Total_Populations,Province,DependencyRatio
0,2013,2.208311e+06,9.490988e+06,9.426862e+06,3.670468e+07,6.123509e+06,6.395435e+07,6.512472e+07,ทั่วประเทศ,0.386348
1,2014,2.232291e+06,9.540494e+06,9.497714e+06,3.666452e+07,6.124853e+06,6.405987e+07,6.498266e+07,ทั่วประเทศ,0.387712
2,2015,2.173518e+06,9.443150e+06,9.354376e+06,3.688957e+07,6.397649e+06,6.425826e+07,6.537933e+07,ทั่วประเทศ,0.389550
3,2016,2.035985e+06,9.408476e+06,9.198821e+06,3.715103e+07,6.672709e+06,6.446703e+07,6.613108e+07,ทั่วประเทศ,0.390879
4,2017,1.955335e+06,9.358071e+06,8.980469e+06,3.739005e+07,6.931164e+06,6.461509e+07,6.625162e+07,ทั่วประเทศ,0.393452


In [9]:
ExpoSmoothPredTotal.to_csv('ExpoSmoothPredTotal.csv',index=False)

In [10]:
reverse_pivot = ExpoSmoothPredTotal.melt(id_vars=['year','Province'], var_name='Age Gruop', value_name='Population')
reverse_pivot

,year,Province,Age Gruop,Population
0,2013,ทั่วประเทศ,Infant,2.208311e+06
1,2014,ทั่วประเทศ,Infant,2.232291e+06
2,2015,ทั่วประเทศ,Infant,2.173518e+06
3,2016,ทั่วประเทศ,Infant,2.035985e+06
4,2017,ทั่วประเทศ,Infant,1.955335e+06
...,...,...,...,...
23707,2046,นราธิวาส,DependencyRatio,3.772641e-01
23708,2047,นราธิวาส,DependencyRatio,3.737277e-01
23709,2048,นราธิวาส,DependencyRatio,3.702454e-01
23710,2049,นราธิวาส,DependencyRatio,3.668159e-01


In [11]:
reverse_pivot.to_csv('ExpoSmoothPredTotalRevPivot.csv',index=False)